In [1]:
import numpy as np
import pandas as pd
import cv2
import face_recognition
import firebase_admin
from firebase_admin import credentials, db
import urllib.request
import os
from datetime import datetime

Firebase set up

In [2]:
cred = credentials.Certificate("C:\\Users\\prana\\OneDrive\\Desktop\\ATTENDANCE\\iotproject.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://iotproject-73165-default-rtdb.firebaseio.com/'
})

In [3]:
path = r'C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder'
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if os.path.isdir(folder_path):
        files = gb.glob(os.path.join(folder_path))
        print(f'For testing data, found {len(files)} in folder {folder}')
    else:
        print(f'{folder} is not a directory in {path}')

Dhoni.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Downey.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Ellon.jfif is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Kavin.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Navaneeth.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Pranavadhar.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Pranavsuriya.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Rohit.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Trump.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder
Usain.jpg is not a directory in C:\Users\prana\OneDrive\Desktop\ATTENDANCE\image_folder


In [4]:
url = 'http://192.168.23.162/cam-lo.jpg'

Checking Attendance.csv

In [5]:
attendance_path = os.path.join(os.getcwd(), 'Attendance.csv')
if os.path.exists(attendance_path):
    os.remove(attendance_path)
df = pd.DataFrame(list())
df.to_csv(attendance_path, index=False)


In [6]:
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['Dhoni.jpg', 'Downey.jpg', 'Ellon.jfif', 'Kavin.jpg', 'Navaneeth.jpg', 'Pranavadhar.jpg', 'Pranavsuriya.jpg', 'Rohit.jpg', 'Trump.jpg', 'Usain.jpg']


In [7]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Dhoni', 'Downey', 'Ellon', 'Kavin', 'Navaneeth', 'Pranavadhar', 'Pranavsuriya', 'Rohit', 'Trump', 'Usain']


Encoding of face 

In [8]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

Marking the attendance

In [9]:
def markAttendance(name):
    now = datetime.now()
    dtString = now.strftime('%H:%M:%S')
    with open("Attendance.csv", 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        if name not in nameList:
            now = datetime.now()
            current_date = now.strftime('%Y-%m-%d')
            f.writelines(f'\n{name},{dtString},{current_date}')
            # Push to Firebase
            ref = db.reference('attendance')
            ref.push({
                'name': name,
                'time': dtString,
                'date': current_date
            })

encodeListKnown = findEncodings(images)
print('Encoding Complete')

Encoding Complete


Image annotation

In [10]:
while True:
    img_resp = urllib.request.urlopen(url)
    imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
    img = cv2.imdecode(imgnp, -1)
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
        else:
            name = "Unknown User"
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    # Optionally, you can mark attendance for unknown users here

    cv2.imshow('Webcam', img)
    key = cv2.waitKey(5)
    if key == ord('q'):
        break
cv2.destroyAllWindows()



URLError: <urlopen error [WinError 10065] A socket operation was attempted to an unreachable host>